# Homework and bake-off: pragmatic color descriptions

In [1]:
__author__ = "Christopher Potts"
__version__ = "CS224u, Stanford, Spring 2020"

## Contents

1. [Overview](#Overview)
1. [Set-up](#Set-up)
1. [All two-word examples as a dev corpus](#All-two-word-examples-as-a-dev-corpus)
1. [Dev dataset](#Dev-dataset)
1. [Random train–test split for development](#Random-train–test-split-for-development)
1. [Question 1: Improve the tokenizer [1 point]](#Question-1:-Improve-the-tokenizer-[1-point])
1. [Use the tokenizer](#Use-the-tokenizer)
1. [Question 2: Improve the color representations [1 point]](#Question-2:-Improve-the-color-representations-[1-point])
1. [Use the color representer](#Use-the-color-representer)
1. [Initial model](#Initial-model)
1. [Question 3: GloVe embeddings [1 points]](#Question-3:-GloVe-embeddings-[1-points])
1. [Try the GloVe representations](#Try-the-GloVe-representations)
1. [Question 4: Color context [3 points]](#Question-4:-Color-context-[3-points])
1. [Your original system [3 points]](#Your-original-system-[3-points])
1. [Bakeoff [1 point]](#Bakeoff-[1-point])

## Overview

This homework and associated bake-off are oriented toward building an effective system for generating color descriptions that are pragmatic in the sense that they would help a reader/listener figure out which color was being referred to in a shared context consisting of a target color (whose identity is known only to the describer/speaker) and a set of distractors.

The notebook [colors_overview.ipynb](colors_overview.ipynb) should be studied before work on this homework begins. That notebook provides backgroud on the task, the dataset, and the modeling code that you will be using and adapting.

The homework questions are more open-ended than previous ones have been. Rather than asking you to implement pre-defined functionality, they ask you to try to improve baseline components of the full system in ways that you find to be effective. As usual, this culiminates in a prompt asking you to develop a novel system for entry into the bake-off. In this case, though, the work you do for the homework will likely be directly incorporated into that system.

## Set-up

See [colors_overview.ipynb](colors_overview.ipynb) for set-up in instructions and other background details.

In [2]:
from colors import ColorsCorpusReader
import os
from sklearn.model_selection import train_test_split
from torch_color_describer import (
    ContextualColorDescriber, create_example_dataset)
import utils
from utils import START_SYMBOL, END_SYMBOL, UNK_SYMBOL

In [3]:
utils.fix_random_seeds()

In [4]:
COLORS_SRC_FILENAME = os.path.join(
    "data", "colors", "filteredCorpus.csv")

## All two-word examples as a dev corpus

So that you don't have to sit through excessively long training runs during development, I suggest working with the two-word-only subset of the corpus until you enter into the late stages of system testing.

In [5]:
dev_corpus = ColorsCorpusReader(
    COLORS_SRC_FILENAME, 
    word_count=2, 
    normalize_colors=True)

In [6]:
dev_examples = list(dev_corpus.read())

This subset has about one-third the examples of the full corpus:

In [7]:
len(dev_examples)

13890

We __should__ worry that it's not a fully representative sample. Most of the descriptions in the full corpus are shorter, and a large proportion are longer. So this dataset is mainly for debugging, development, and general hill-climbing. All findings should be validated on the full dataset at some point.

## Dev dataset

The first step is to extract the raw color and raw texts from the corpus:

In [8]:
dev_rawcols, dev_texts = zip(*[[ex.colors, ex.contents] for ex in dev_examples])

The raw color representations are suitable inputs to a model, but the texts are just strings, so they can't really be processed as-is. Question 1 asks you to do some tokenizing!

## Random train–test split for development

For the sake of development runs, we create a random train–test split:

In [9]:
dev_rawcols_train, dev_rawcols_test, dev_texts_train, dev_texts_test = \
    train_test_split(dev_rawcols, dev_texts)

## Question 1: Improve the tokenizer [1 point]

This is the first required question – the first required modification to the default pipeline.

The function `tokenize_example` simply splits its string on whitespace and adds the required start and end symbols:

In [10]:
from nltk.tokenize import TreebankWordTokenizer
'''
An inspection of the words in the contexts reveals the following properties:
1. There are quite many spelling mistakes as is common in a chat like environment
2. There is extensive use of comparatives and superlatives
3. There are a lot of short forms used

Based on these observations, a word tokenizer does not make sense. Hence we use a pretrained wordpiece tokenizer
'''

from transformers import BertTokenizer
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_example(s):
    return [START_SYMBOL] + bert_tokenizer.tokenize(s) + [END_SYMBOL]

In [11]:
tokenize_example(dev_texts_train[376])

['<s>', 'aqua', ',', 'tea', '##l', '</s>']

__Your task__: Modify `tokenize_example` so that it does something more sophisticated with the input text. 

__Notes__:

* There are useful ideas for this in [Monroe et al. 2017](https://transacl.org/ojs/index.php/tacl/article/view/1142)
* There is no requirement that you do word-level tokenization. Sub-word and multi-word are options.
* This question can interact with the size of your vocabulary (see just below), and in turn with decisions about how to use `UNK_SYMBOL`.

__Important__: don't forget to add the start and end symbols, else the resulting models will definitely be terrible!

## Use the tokenizer

Once the tokenizer is working, run the following cell to tokenize your inputs:

In [12]:
dev_seqs_train = [tokenize_example(s) for s in dev_texts_train]

dev_seqs_test = [tokenize_example(s) for s in dev_texts_test]

We use only the train set to derive a vocabulary for the model:

In [13]:
dev_vocab = sorted({w for toks in dev_seqs_train for w in toks}) + [UNK_SYMBOL]

It's important that the `UNK_SYMBOL` is included somewhere in this list. Test examples with word not seen in training will be mapped to `UNK_SYMBOL`. If you model's vocab is the same as your train vocab, then `UNK_SYMBOL` will never be encountered during training, so it will be a random vector at test time.

In [14]:
len(dev_vocab)

963

## Question 2: Improve the color representations [1 point]

This is the second required pipeline improvement for the assignment. 

The following functions do nothing at all to the raw input colors we get from the corpus. 

In [15]:
import numpy as np
def represent_color_context(colors):
    
    # Improve me!
    '''
    use the differenc between the target and distractors
    '''
    colors = np.array(colors)
    target_color = colors[-1]
    
    colors = [target_color - colors[0], target_color- colors[1], target_color]
    return [represent_color(color) for color in colors]


def represent_color(color):

    # Improve me!
    '''
    Use fft transform of the color as in Monroe
    '''
    h, s, v = color
    color_ft = []
    for j in range(3):
        for k in range(3):
            for l in range(3):
                color_ft.append(np.exp(-2j * np.pi * (j * h + k*s/2 + l*v/2)))
    color = np.concatenate([np.real(color_ft), np.imag(color_ft)])
    
    return color

In [16]:
represent_color_context(dev_rawcols_train[0])

[array([ 1.        , -0.36812455, -0.72896863,  1.        , -0.36812455,
        -0.72896863,  1.        , -0.36812455, -0.72896863,  0.88294759,
        -0.76153831, -0.3222657 ,  0.88294759, -0.76153831, -0.3222657 ,
         0.88294759, -0.76153831, -0.3222657 ,  0.5591929 , -0.97667228,
         0.15988119,  0.5591929 , -0.97667228,  0.15988119,  0.5591929 ,
        -0.97667228,  0.15988119,  0.        , -0.92977649,  0.68454711,
         0.        , -0.92977649,  0.68454711,  0.        , -0.92977649,
         0.68454711, -0.46947156, -0.6481199 ,  0.94664926, -0.46947156,
        -0.6481199 ,  0.94664926, -0.46947156, -0.6481199 ,  0.94664926,
        -0.82903757, -0.21473533,  0.98713627, -0.82903757, -0.21473533,
         0.98713627, -0.82903757, -0.21473533,  0.98713627]),
 array([ 1.        ,  0.12533323, -0.96858316,  1.        ,  0.12533323,
        -0.96858316,  1.        ,  0.12533323, -0.96858316, -0.98768834,
         0.03141076,  0.99556196, -0.98768834,  0.03141076,  0

__Your task__: Modify `represent_color_context` and/or `represent_color` to represent colors in a new way.
    
__Notes__:

* The Fourier-transform method of [Monroe et al. 2017](https://transacl.org/ojs/index.php/tacl/article/view/1142) is a proven choice.
* You are not required to keep `represent_color`. This might be unnatural if you want to perform an operation on each color trio all at once.
* For that matter, if you want to process all of the color contexts in the entire data set all at once, that is fine too, as long as you can also perform the operation at test time with an unknown number of examples being tested.

## Use the color representer

The following cell just runs your `represent_color_context` on the train and test sets:

In [17]:
dev_cols_train = [represent_color_context(colors) for colors in dev_rawcols_train]

dev_cols_test = [represent_color_context(colors) for colors in dev_rawcols_test]

At this point, our preprocessing steps are complete, and we can fit a first model.

## Initial model

The first model is configured right now to be a small model run for just a few iterations. It should be enough to get traction, but it's unlikely to be a great model. You are free to modify this configuration if you wish; it is here just for demonstration and testing:

In [45]:
dev_mod = ContextualColorDescriber(
    dev_vocab, 
    embed_dim=10, 
    hidden_dim=10, 
    max_iter=5, 
    batch_size=128)

In [46]:
_ = dev_mod.fit(dev_cols_train, dev_seqs_train)

Epoch 5; err = 120.32878661155702

As discussed in [colors_overview.ipynb](colors_overview.ipynb), our primary metric is `listener_accuracy`:

In [47]:
dev_mod.listener_accuracy(dev_cols_test, dev_seqs_test)

0.7975813417794414

We can also see the model's predicted sequences given color context inputs:

In [21]:
dev_mod.predict(dev_cols_test[1:10])

[['<s>', 'yellow', 'green', '</s>'],
 ['<s>', 'purple', 'purple', '</s>'],
 ['<s>', 'bright', 'pink', '</s>'],
 ['<s>', 'bright', 'green', '</s>'],
 ['<s>', 'bright', 'pink', '</s>'],
 ['<s>', 'purple', 'purple', '</s>'],
 ['<s>', 'purple', 'purple', '</s>'],
 ['<s>', 'bright', 'purple', '</s>'],
 ['<s>', 'purple', 'purple', '</s>']]

In [22]:
dev_seqs_test[1:10]

[['<s>', 'darker', 'yellow', '</s>'],
 ['<s>', 'dull', 'one', '</s>'],
 ['<s>', 'light', 'pink', '</s>'],
 ['<s>', 'like', 'mint', '</s>'],
 ['<s>', 'bright', 'pink', '.', '</s>'],
 ['<s>', 'dull', '##er', 'purple', '</s>'],
 ['<s>', 'dull', 'purple', '</s>'],
 ['<s>', 'bright', 'purple', '</s>'],
 ['<s>', 'brighter', 'purple', '</s>']]

In [25]:
dev_mod = ContextualColorDescriber(
    dev_vocab, 
    embed_dim=10, 
    hidden_dim=10, 
    max_iter=5, 
    batch_size=128)

best_mod = utils.fit_classifier_with_crossvalidation(
    dev_cols_train, 
    dev_seqs_train, 
    dev_mod, 
    cv=2,
    scoring=None,
    param_grid={'hidden_dim': [10, 20, 64, 128], "max_iter": [5, 10]})

Epoch 5; err = 95.718495488166814

Best params: {'hidden_dim': 64, 'max_iter': 5}
Best score: 0.827


In [48]:
best_mod.listener_accuracy(dev_cols_test, dev_seqs_test)

0.835588828102505

In [27]:
best_mod.predict(dev_cols_test[1:10])

[['<s>', 'dull', 'green', '</s>'],
 ['<s>', 'dull', 'pink', '</s>'],
 ['<s>', 'hot', 'pink', '</s>'],
 ['<s>', 'mint', 'green', '</s>'],
 ['<s>', 'hot', 'pink', '</s>'],
 ['<s>', 'dark', 'purple', '</s>'],
 ['<s>', 'dark', 'purple', '</s>'],
 ['<s>', 'bright', 'purple', '</s>'],
 ['<s>', 'bright', 'purple', '</s>']]

## Question 3: GloVe embeddings [1 points]

The above model uses a random initial embedding, as configured by the decoder used by `ContextualColorDescriber`. This homework question asks you to consider using GloVe inputs. 

__Your task__: Complete `create_glove_embedding` so that it creates a GloVe embedding based on your model vocabulary. This isn't mean to be analytically challenging, but rather just to create a basis for you to try out other kinds of rich initialization.

In [49]:
GLOVE_HOME = os.path.join('data', 'glove.6B')

In [50]:
def create_glove_embedding(vocab, glove_base_filename='glove.6B.50d.txt'):
    
    # Use `utils.glove2dict` to read in the GloVe file:    
    ##### YOUR CODE HERE
    glove_src = os.path.join(GLOVE_HOME, glove_base_filename)
    GLOVE = utils.glove2dict(glove_src)
    
    # Use `utils.create_pretrained_embedding` to create the embedding.
    # This function will, by default, ensure that START_TOKEN, 
    # END_TOKEN, and UNK_TOKEN are included in the embedding.
    ##### YOUR CODE HERE
    embedding, vocab = utils.create_pretrained_embedding(GLOVE, vocab)

    
    # Be sure to return the embedding you create as well as the
    # vocabulary returned by `utils.create_pretrained_embedding`,
    # which is likely to have been modified from the input `vocab`.
    
    ##### YOUR CODE HERE

    return embedding, vocab

## Try the GloVe representations

Let's see if GloVe helped for our development data:

In [51]:
dev_glove_embedding, dev_glove_vocab = create_glove_embedding(dev_vocab)

The above might dramatically change your vocabulary, depending on how many items from your vocab are in the Glove space:

In [52]:
len(dev_vocab)

963

In [53]:
len(dev_glove_vocab)

963

In [66]:
dev_mod_glove = ContextualColorDescriber(
    dev_glove_vocab, 
    embedding=dev_glove_embedding,
    hidden_dim=10, 
    max_iter=5, 
    batch_size=128)

In [67]:
_ = dev_mod_glove.fit(dev_cols_train, dev_seqs_train)

Epoch 5; err = 120.82809722423553

In [68]:
dev_mod_glove.listener_accuracy(dev_cols_test, dev_seqs_test)

0.7898070832133602

In [61]:
dev_mod_glove = ContextualColorDescriber(
    dev_glove_vocab, 
    embedding=dev_glove_embedding,
    hidden_dim=10, 
    max_iter=5, 
    batch_size=128)

best_mod_glove = utils.fit_classifier_with_crossvalidation(
    dev_cols_train, 
    dev_seqs_train, 
    dev_mod_glove, 
    cv=2,
    scoring=None,
    param_grid={'hidden_dim': [10, 20, 64], "max_iter": [5, 10]})

Epoch 5; err = 96.2347082495689435

Best params: {'hidden_dim': 64, 'max_iter': 5}
Best score: 0.821


In [62]:
best_mod_glove.listener_accuracy(dev_cols_test, dev_seqs_test)

0.800460696803916

You probably saw a small boost, assuming your tokeization scheme leads to good overlap with the GloVe vocabulary. The input representations are larger than in our previous model (at least as I configured things), so we would need to do more runs with higher `max_iter` values to see whether this is worthwhile overall.

## Question 4: Color context [3 points]

The final required homework question is the most challenging, but it should set you up to think in much more flexible ways about the underlying model we're using.

The question asks you to modify various model components in `torch_color_describer.py`. The section called [Modifying the core model](colors_overview.ipynb#Modifying-the-core-model) from the core unit notebook provides a number of examples illustrating the basic techniques, so you might review that material if you get stuck here.

__Your task__: [Monroe et al. 2017](https://transacl.org/ojs/index.php/tacl/article/view/1142) append the target color (the final one in the context) to each input token that gets processed by the decoder. The question asks you to subclass the `Decoder` and `EncoderDecoder` from `torch_color_describer.py` so that you can build models that do this.

__Step 1__: Modify the `Decoder` so that the input vector to the model at each timestep is not just a token representaton `x` but the concatenation of `x` with the representation of the target color.

__Notes__:

* You might notice at this point that the original `Decoder.forward` method has an optional keyword argument `target_colors` that is passed to `Decoder.get_embeddings`. Because this is already in place, all you have to do is modify the `get_embeddings` method to use this argument.

* The change affects the configuration of `self.rnn`, so you need to subclass the `__init__` method as well, so that its `input_size` argument accomodates the embedding as well as the color representations.

* You can do the relevant operations efficiently in pure PyTorch using `repeat_interleave` and `cat`, but the important thing is to get a working implementation – you can always optimize the code later if the ideas prove useful to you. 

Here's skeleton code for you to flesh out:

In [69]:
import torch
import torch.nn as nn
from torch_color_describer import Decoder

class ColorContextDecoder(Decoder):    
    def __init__(self, color_dim, *args, **kwargs):
        self.color_dim = color_dim
        super().__init__(*args, **kwargs)
        
        # Fix the `self.rnn` attribute:
        ##### YOUR CODE HERE

        self.rnn = nn.GRU(
            input_size=self.embed_dim + self.color_dim,
            hidden_size=self.hidden_dim,
            batch_first=True)
        

    def get_embeddings(self, word_seqs, target_colors=None):  
        """You can assume that `target_colors` is a tensor of shape 
        (m, n), where m is the length of the batch (same as 
        `word_seqs.shape[0]`) and n is the dimensionality of the 
        color representations the model is using. The goal is
        to attached each color vector i to each of the tokens in
        the ith sequence of (the embedded version of) `word_seqs`.
        
        """        
        ##### YOUR CODE HERE
        
        word_emb = self.embedding(word_seqs)
        tc = target_colors.unsqueeze(1)
        tc = torch.repeat_interleave(tc, repeats=word_emb.shape[1], dim=1)
        word_emb_combined = torch.cat((word_emb, tc), 2)
        return word_emb_combined



__Step 2__: Modify the `EncoderDecoder`. For this, you just need to make a small change to the `forward` method: extract the target colors from `color_seqs` and feed them to the decoder.

In [70]:
from torch_color_describer import EncoderDecoder

class ColorizedEncoderDecoder(EncoderDecoder):
    
    def forward(self, 
            color_seqs, 
            word_seqs, 
            seq_lengths=None, 
            hidden=None, 
            targets=None):
        if hidden is None:
            hidden = self.encoder(color_seqs)
            
        # Extract the target colors from `color_seqs` and 
        # feed them to the decoder, which already has a
        # `target_colors` keyword.        
        ##### YOUR CODE HERE
        target_colors = color_seqs[:, -1, :]
        output, hidden = self.decoder(word_seqs, seq_lengths=seq_lengths, hidden=hidden, target_colors=target_colors)
        
        return output, hidden, targets

__Step 3__: Finally, as in the examples in [Modifying the core model](colors_overview.ipynb#Modifying-the-core-model), you need to modify the `build_graph` method of `ContextualColorDescriber` so that it uses your new `ColorContextDecoder` and `ColorizedEncoderDecoder`. Here's starter code:

In [71]:
from torch_color_describer import Encoder

class ColorizedInputDescriber(ContextualColorDescriber):
        
    def build_graph(self):
        
        # We didn't modify the encoder, so this is
        # just copied over from the original:
        encoder = Encoder(
            color_dim=self.color_dim,
            hidden_dim=self.hidden_dim)

        # Use your `ColorContextDecoder`, making sure
        # to pass in all the keyword arguments coming
        # from `ColorizedInputDescriber`:
        
        ##### YOUR CODE HERE
        decoder = ColorContextDecoder(
            color_dim=self.color_dim,
            vocab_size=self.vocab_size,
            embed_dim=self.embed_dim,
            embedding=self.embedding,
            hidden_dim=self.hidden_dim)

        
        # Return a `ColorizedEncoderDecoder` that uses
        # your encoder and decoder:
        
        ##### YOUR CODE HERE

        return ColorizedEncoderDecoder(encoder, decoder)


That's it! Since these modifications are pretty intricate, you might want to use [a toy dataset](colors_overview.ipynb#Toy-problems-for-development-work) to debug it:

In [72]:
toy_color_seqs, toy_word_seqs, toy_vocab = create_example_dataset(
    group_size=50, vec_dim=2)

In [73]:
toy_color_seqs_train, toy_color_seqs_test, toy_word_seqs_train, toy_word_seqs_test = \
    train_test_split(toy_color_seqs, toy_word_seqs)

In [74]:
toy_mod = ColorizedInputDescriber(
    toy_vocab, 
    embed_dim=10, 
    hidden_dim=10, 
    max_iter=100, 
    batch_size=128)

In [75]:
_ = toy_mod.fit(toy_color_seqs_train, toy_word_seqs_train)

Epoch 100; err = 0.15312781929969788

In [76]:
toy_mod.listener_accuracy(toy_color_seqs_test, toy_word_seqs_test)

1.0

If that worked, then you can now try this model on SCC problems!

In [80]:
dev_mod_colorized = ColorizedInputDescriber(
    dev_glove_vocab, 
    embedding=dev_glove_embedding,
    hidden_dim=10, 
    max_iter=5, 
    batch_size=128)

In [81]:
_ = dev_mod_colorized.fit(dev_cols_train, dev_seqs_train)

Epoch 5; err = 121.03712475299835

In [82]:
dev_mod_colorized.listener_accuracy(dev_cols_test, dev_seqs_test)

0.8767636049524906

In [84]:
dev_mod_colorized = ColorizedInputDescriber(
    dev_glove_vocab, 
    embedding=dev_glove_embedding,
    hidden_dim=10, 
    max_iter=5, 
    batch_size=128)

best_mod_colorized = utils.fit_classifier_with_crossvalidation(
    dev_cols_train, 
    dev_seqs_train, 
    dev_mod_colorized, 
    cv=2,
    scoring=None,
    param_grid={'hidden_dim': [10, 20, 64], 'max_iter': [5, 10]})

Epoch 5; err = 115.913144707679755

Best params: {'hidden_dim': 10, 'max_iter': 5}
Best score: 0.898


In [85]:
best_mod_colorized.listener_accuracy(dev_cols_test, dev_seqs_test)

0.8738842499280162

## Your original system [3 points]

There are many options for your original system, which consists of the full pipeline – all preprocessing and modeling steps. You are free to use any model you like, as long as you subclass `ContextualColorDescriber` in a way that allows its `listener_accuracy` method to behave in the expected way.

So that we can evaluate models in a uniform way for the bake-off, we ask that you modify the function `my_original_system` below so that it accepts a trained instance of your model and does any preprocessing steps required by your model.

If we seek to reproduce your results, we will rerun this entire notebook. Thus, it is fine if your `my_original_system` makes use of functions you wrote or modified above this cell.

In [86]:
def my_original_system(trained_model, color_seqs_test, texts_test): 
    """Feel free to modify this code to accommodate the needs of
    your system. Just keep in mind that it will get raw corpus
    examples as inputs for the bake-off.
    
    """    
    # `word_seqs_test` is a list of strings, so tokenize each of
    # its elements:    
    tok_seqs = [tokenize_example(s) for s in texts_test]
    
    col_seqs = [represent_color_context(colors) 
                for colors in color_seqs_test]

    # Return the `listener_accuracy` for your model:
    return trained_model.listener_accuracy(col_seqs, tok_seqs)

If `my_original_system` works on test sets you create from the corpus distribution, then it will works for the bake-off, so consider checking that. For example, this would check that `dev_mod` above passes muster:

In [162]:
my_original_system(dev_mod, dev_rawcols_test, dev_texts_test)

0.7975813417794414

In the cell below, please provide a brief technical description of your original system, so that the teaching team can gain an understanding of what it does. This will help us to understand your code and analyze all the submissions to identify patterns and strategies.

# Enter your system description in this cell.

The different components of the original system is  described below:

1. **Tokenizer**: We use a pretrained BERT tokenizer to tokenize the input. We have also tried NLTK's treebank tokenizer and found the BERT tokenizer to be better. We also used a BERT WordPiece tokenizer trained on the train set but it performed comparable or less than the pretrained tokenizer and hence we finally settles on that. The subword tokenizer are expected to perform better as the input text is conversation-like and hence filled with spelling errors, letter repetitions, etc.
2. **Word embeddings**: We use 50-dim glove vectors for our embeddings. Though due to the use of subword tokenization, there will be lot of words not in the vocabulary of the Glove, but we expect to find the frequent words in the vocabulary and only the word-endings, comparative/superlative modiffier wont be in vocab which we can assign randomly. We could have also used some pretrained BPE embeddings but did not experiment with it.
3. **Color Representation**: For indivdual color representation, we use the 54-dim representation of the HSV values using FFT following the method described in Monroe et al. For representing the sequence of colors, we subtract the target color from the distractors before calculating the HSV. We believe that this is helpful for the model because the description of the target depends on how close or far the distractors are rathet than what color the distractor is.
4. **Encoder**: We experimented with different encoders:
    i. Bidirectional RNN encoder: The encoder generates a concatenated representation of the target color from both directions.
    ii. Fully Connetced Encoder: The encoder generates a linear transformation of the concatenated colors representations.
    iii. Convolution Encoder: The encoder applies a 1-d convolution with kernel size of 3 and no padding to generate a representation to be fed to the decoder.
    We find through the experiments below that the bidirectional encoder does not perform better than the forward RNN ecnoder used above. The fully connected encoder and convolution encoder is comparable in function and performance and performs best of the task. The FC Encoder achieves a score of 95.85% on the dev set (with <= 2 words).
5. **Decoder**: We find from the Q4 that the colorized decoder performs much better than the non-colorized one and hence we use the same for all the different encoders.
6. **Hidden Dimensiom**: Based on the cross validation results in the questions of the HW above, we used the hidden dimensions of 10 for the different models below (we also cross validated the results and found the hidden size of 10 to work the best) and found that the system with FC Encoder and Conv Encoder words eqaully better.
7. **Train epochs**: We trained all models for 5 epochs (we also tried training for 10 epochs but found that training for 5 epochs was better in most cases)

Hence, in summary the components of the best original system is a model with Fully Connetced Encoder/Conv Encoder and hidden dimension of 10 and trained for 5 epochs. We report all scores below for the Conv encoder with 10 hidden units trained for 5 epochs. The results of other configurations can be found in the notebook's output.

The peak score on the dev test set was 93.72 and on the full dataset was 96.96
We report the score on the full dataset below.

My peak score was: 0.9696


In [94]:
from torch_color_describer import Encoder, Decoder
class BiDirEncoder(Encoder):
    """Simple Encoder model based on a GRU cell.

    Parameters
    ----------
    color_dim : int
    hidden_dim : int

    """
    def __init__(self, *args, **kwargs):
        super(BiDirEncoder, self).__init__(*args, **kwargs)
        self.rnn = nn.GRU(
            input_size=self.color_dim,
            hidden_size=self.hidden_dim,
            bidirectional=True, 
            batch_first=True)

    def forward(self, color_seqs):
        output, hidden = self.rnn(color_seqs)
        fwd_hidden = hidden[0:hidden.size(0):2]
        bwd_hidden = hidden[1:hidden.size(0):2]
        result= torch.cat([fwd_hidden, bwd_hidden], dim=2)
        return result
    
from torch_color_describer import EncoderDecoder

class BidirColorizedColorDescriber(ContextualColorDescriber): 
                
    def build_graph(self):
        encoder = BiDirEncoder(
            color_dim=self.color_dim,
            hidden_dim=self.hidden_dim) 

        decoder = ColorContextDecoder(
            color_dim=self.color_dim,
            vocab_size=self.vocab_size,
            embed_dim=self.embed_dim,
            embedding=self.embedding,
            hidden_dim=self.hidden_dim*2) 

        return ColorizedEncoderDecoder(encoder, decoder)

In [111]:
dev_mod_bidir = BidirColorizedColorDescriber(
    dev_glove_vocab, 
    embedding=dev_glove_embedding,
    hidden_dim=10, 
    max_iter=5, 
    batch_size=128)
_ = dev_mod_bidir.fit(dev_cols_train, dev_seqs_train)
dev_mod_bidir.listener_accuracy(dev_cols_test, dev_seqs_test)

Epoch 5; err = 107.42708611488342

0.8491217967175353

In [117]:
# FC encoder
import torch.nn.functional as F
class FCEncoder(Encoder):
    """Simple Encoder model based on a GRU cell.

    Parameters
    ----------
    color_dim : int
    hidden_dim : int

    """
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.fc = nn.Linear(self.color_dim*3, self.hidden_dim)

    def forward(self, color_seqs):
        n = color_seqs.size(0)
        h = self.fc(color_seqs.view(n, -1))
        return h.unsqueeze(dim=0)
    
    
class FCColorDescriber(ContextualColorDescriber): 
                
    def build_graph(self):
        encoder = FCEncoder(
            color_dim=self.color_dim,
            hidden_dim=self.hidden_dim) 

        decoder = ColorContextDecoder(
            color_dim=self.color_dim,
            vocab_size=self.vocab_size,
            embed_dim=self.embed_dim,
            embedding=self.embedding,
            hidden_dim=self.hidden_dim)
        return ColorizedEncoderDecoder(encoder, decoder)

In [151]:
dev_mod_fc = FCColorDescriber(
    dev_glove_vocab, 
    embedding=dev_glove_embedding,
    hidden_dim=10, 
    max_iter=5, 
    batch_size=128)
_ = dev_mod_fc.fit(dev_cols_train, dev_seqs_train)
dev_mod_fc.listener_accuracy(dev_cols_test, dev_seqs_test)

Epoch 5; err = 115.85484743118286

0.9260005758710049

In [128]:
# Conv encoder
import torch.nn.functional as F
class ConvEncoder(Encoder):
    """Simple Encoder model based on a GRU cell.

    Parameters
    ----------
    color_dim : int
    hidden_dim : int

    """
    def __init__(self, *args, padding=0, **kwargs):
        super().__init__(*args, **kwargs)
        self.padding = padding
        self.conv1 = nn.Conv1d(self.color_dim, self.hidden_dim, kernel_size=3, padding=self.padding)

    def forward(self, color_seqs):
        h = self.conv1(color_seqs.permute(0, 2, 1))
        hidden, _ = torch.max(h, dim=-1)
        return hidden.unsqueeze(dim=0)
    
    
class ConvColorDescriber(ContextualColorDescriber): 
    def __init__(self, *args, padding=0, **kwargs):
        self.padding = padding
        super().__init__(*args, **kwargs)
        
                
    def build_graph(self):
        encoder = ConvEncoder(
            color_dim=self.color_dim,
            hidden_dim=self.hidden_dim,
            padding=self.padding
        ) 

        decoder = ColorContextDecoder(
            color_dim=self.color_dim,
            vocab_size=self.vocab_size,
            embed_dim=self.embed_dim,
            embedding=self.embedding,
            hidden_dim=self.hidden_dim)
        return ColorizedEncoderDecoder(encoder, decoder)

In [159]:
dev_mod_conv = ConvColorDescriber(
    dev_glove_vocab, 
    embedding=dev_glove_embedding,
    hidden_dim=10, 
    max_iter=5, 
    batch_size=128)
_ = dev_mod_conv.fit(dev_cols_train, dev_seqs_train)
dev_mod_conv.listener_accuracy(dev_cols_test, dev_seqs_test)

Epoch 5; err = 114.16141903400421

0.9372300604664555

In [160]:
best_model_dev = dev_mod_conv

In [146]:
corpus = ColorsCorpusReader(
    COLORS_SRC_FILENAME, 
    word_count=None, 
    normalize_colors=True)

examples = list(corpus.read())

rawcols, texts = zip(*[[ex.colors, ex.contents] for ex in examples])

rawcols_train, rawcols_test, texts_train, texts_test = train_test_split(rawcols, texts)

seqs_train = [tokenize_example(s) for s in texts_train]

seqs_test = [tokenize_example(s) for s in texts_test]

vocab = sorted({w for toks in seqs_train for w in toks}) + [UNK_SYMBOL]


cols_train = [represent_color_context(colors) for colors in rawcols_train]

cols_test = [represent_color_context(colors) for colors in rawcols_test]

glove_embedding, glove_vocab = create_glove_embedding(vocab)

In [147]:
models = {}
    
for model_class in [FCColorDescriber, ConvColorDescriber, ColorizedInputDescriber]: 
    for hidden_dim in [5, 10]:
        for i in range(3):
            model = model_class(
                        glove_vocab, 
                        embedding=glove_embedding,
                        hidden_dim=hidden_dim, 
                        max_iter=5, 
                        batch_size=128)
            _ = model.fit(cols_train, seqs_train)
            acc = model.listener_accuracy(cols_test, seqs_test)
            key = (model_class.__name__, hidden_dim, i, acc)
            print(key)
            models[key] = model

Epoch 5; err = 171.06927654147148

('FCColorDescriber', 5, 0, 0.9312281896331602)


Epoch 5; err = 176.04462772607803

('FCColorDescriber', 5, 1, 0.963316026895906)


Epoch 5; err = 175.79422226548195

...

('ColorizedInputDescriber', 10, 0, 0.9052685334922121)


Epoch 5; err = 157.07487878203392

('ColorizedInputDescriber', 10, 1, 0.8890118307941102)


Epoch 5; err = 151.92403364181519

('ColorizedInputDescriber', 10, 2, 0.945442165290663)


In [156]:
best_model = models[('ConvColorDescriber', 10, 0, 0.9696995488977785)]

In [161]:
my_original_system(best_model_dev, dev_rawcols_test, dev_texts_test)

0.9372300604664555

In [158]:
my_original_system(best_model, rawcols_test, texts_test)

0.9696995488977785

In [ ]:
# Please do not remove this comment.

In [165]:
# train a model on whole data for bakeoff
model = ConvColorDescriber(
        glove_vocab, 
        embedding=glove_embedding,
        hidden_dim=10, 
        max_iter=5, 
        batch_size=128)
_ = model.fit([represent_color_context(colors) for colors in rawcols], [tokenize_example(s) for s in texts])

Epoch 5; err = 202.32969163358212

## Bakeoff [1 point]

For the bake-off, we will release a test set. The announcement will go out on the discussion forum. You will evaluate your custom model from the previous question on these new datasets using your `my_original_system` function. Rules:

1. Only one evaluation is permitted.
1. No additional system tuning is permitted once the bake-off has started.

The cells below this one constitute your bake-off entry.

People who enter will receive the additional homework point, and people whose systems achieve the top score will receive an additional 0.5 points. We will test the top-performing systems ourselves, and only systems for which we can reproduce the reported results will win the extra 0.5 points.

Late entries will be accepted, but they cannot earn the extra 0.5 points. Similarly, you cannot win the bake-off unless your homework is submitted on time.

The announcement will include the details on where to submit your entry.

In [166]:
# Enter your bake-off assessment code in this cell.
# Please do not remove this comment.

COLORS_BAKEOFF_SRC_FILENAME = os.path.join(
    "data", "colors", "cs224u-colors-bakeoff-data.csv")

bakeoff_corpus = ColorsCorpusReader(COLORS_BAKEOFF_SRC_FILENAME)

# This code just extracts the colors and texts from the new corpus:
bakeoff_rawcols, bakeoff_texts = zip(*[
    [ex.colors, ex.contents] for ex in bakeoff_corpus.read()])

# Original system function call; `my_mod` is your ***trained*** model:
my_original_system(model, bakeoff_rawcols, bakeoff_texts)



0.9940915805022157

In [167]:
# On an otherwise blank line in this cell, please enter
# your listener_accuracy score as reported by the code
# above. Please enter only a number between 0 and 1 inclusive. 
# Please do not remove this comment.
0.9940915805022157

0.9940915805022157